In [1]:
### Libraries

import math, cv2
import glob, os, time
import zipfile
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
# Unzip all files 

# curiosis_zip = zipfile.ZipFile('/home/data/04_SSD4TB/hwcho/001_cancer/230811_debug_stitching.zip')
# tqdm(curiosis_zip.extractall('/home/data/04_SSD4TB/hwcho/001_cancer/230811_debug_stitching/'))

# curiosis_zip = zipfile.ZipFile('/home/data/04_SSD4TB/hwcho/001_cancer/230811_debug_stitching/10.zip')
# tqdm(curiosis_zip.extractall('/home/data/04_SSD4TB/hwcho/001_cancer/230811_debug_stitching'))

# curiosis_zip = zipfile.ZipFile('/home/data/04_SSD4TB/hwcho/001_cancer/230811_debug_stitching/14.zip')
# tqdm(curiosis_zip.extractall('/home/data/04_SSD4TB/hwcho/001_cancer/230811_debug_stitching'))

In [2]:
# test01  : to filter out the images with 'small tissue included'

test01_root = 'home/hwcho/001_cancer/[test]curiosis/230811_debug_stitching/10/'
test01_root = '10'

In [3]:
scan_areas01 = sorted(glob.glob(os.path.join(test01_root, 'ScanArea**')))
scan_areas01

['10/ScanArea01', '10/ScanArea05', '10/ScanArea06']

In [4]:
test02_root = 'home/hwcho/001_cancer/[test]curiosis/230811_debug_stitching/14/'
test02_root = '14'

In [5]:
scan_areas02 = sorted(glob.glob(os.path.join(test02_root, 'ScanArea**')))
scan_areas02

['14/ScanArea01', '14/ScanArea01.zip']

In [28]:
class area_filter():
    
    def __init__(self, tissue_dir, staining_th, area_th):
        self.tissue_dir = tissue_dir
        self.staining_th = staining_th
        self.area_th = area_th
    
    def _get_tissue_area(self, img_dir):
    
        img_gray = cv2.imread(img_dir, 0)
        _, thr = cv2.threshold(img_gray, self.staining_th, 255, cv2.THRESH_BINARY_INV)
        
        tissue_area = int(np.sum(thr/255))
        total_area = img_gray.shape[0] * img_gray.shape[1]
        
        self.ratio = tissue_area / total_area

    
    def get_outarea_index(self, areas):
        out_index = []

        for area in tqdm(areas) :
            area_exclusion = []
            area_exclusion.append(area)
            test_scan = sorted(glob.glob(os.path.join(area, '**.jpg')))
            for idx, scans in enumerate(test_scan):
                tissue_ratio = self._get_tissue_area(scans)
                if self.ratio < self.area_th:
                    area_exclusion.append(idx)
            out_index.append(area_exclusion)
        
        return out_index
    
    def main(self):
        
        tissue_areas = sorted(glob.glob(os.path.join(self.tissue_dir, 'ScanArea**')))
        tissue_ratio = self.get_outarea_index(tissue_areas)
        
        return tissue_ratio

In [29]:
test_class = area_filter(test01_root, 200, 0.5)

In [31]:
removing_indexes = test_class.main()

3


100%|██████████| 3/3 [00:42<00:00, 14.07s/it]


In [32]:
removing_indexes

[['10/ScanArea01',
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  22,
  23,
  24,
  25,
  26,
  27,
  29,
  30,
  31,
  32,
  36,
  37,
  38,
  39,
  40,
  44,
  45,
  46,
  47,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104],
 ['10/ScanArea05',
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  16,
  17,
  18,
  19,
  22,
  23,
  24,
  25,
  26,
  27,
  29,
  30,
  31,
  36,
  37,
  38,
  39,
  40,
  41,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78

In [18]:
# tissue10_ratio35 = get_outarea_index(scan_areas01, 0.35)
# tissue10_ratio30 = get_outarea_index(scan_areas01, 0.30)
# tissue10_ratio25 = get_outarea_index(scan_areas01, 0.25)

tissue10_ratio05 = get_outarea_index(scan_areas01, 0.05)
tissue10_ratio07 = get_outarea_index(scan_areas01, 0.07)
tissue10_ratio10 = get_outarea_index(scan_areas01, 0.10)
tissue10_ratio15 = get_outarea_index(scan_areas01, 0.15)

100%|████████████████████████████████████████████████████████████| 3/3 [00:39<00:00, 13.08s/it]


In [22]:
with open('tissue10_ratio05.pkl', 'wb') as f:
    pickle.dump(tissue10_ratio05, f)
with open('tissue10_ratio07.pkl', 'wb') as f:
    pickle.dump(tissue10_ratio07, f)
with open('tissue10_ratio10.pkl', 'wb') as f:
    pickle.dump(tissue10_ratio10, f)
with open('tissue10_ratio15.pkl', 'wb') as f:
    pickle.dump(tissue10_ratio15, f)

In [ ]:
# tissue14_ratio35 = get_outarea_index(scan_areas02, 0.35)
# tissue14_ratio30 = get_outarea_index(scan_areas02, 0.30)
# tissue14_ratio25 = get_outarea_index(scan_areas02, 0.25)

tissue14_ratio05 = get_outarea_index(scan_areas02, 0.05)
tissue14_ratio07 = get_outarea_index(scan_areas02, 0.07)
tissue14_ratio10 = get_outarea_index(scan_areas02, 0.10)
tissue14_ratio15 = get_outarea_index(scan_areas02, 0.15)

In [ ]:
with open('tissue14_ratio05.pkl', 'wb') as f:
    pickle.dump(tissue14_ratio05, f)
with open('tissue14_ratio07.pkl', 'wb') as f:
    pickle.dump(tissue14_ratio07, f)
with open('tissue14_ratio10.pkl', 'wb') as f:
    pickle.dump(tissue14_ratio10, f)
with open('tissue14_ratio15.pkl', 'wb') as f:
    pickle.dump(tissue14_ratio15, f)